У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/Machine Learning для людей/data/customer/customer_segmentation_train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
print("Розмір датасету:", df.shape)
print("Пропуски до обробки:\n", df.isnull().sum())

Розмір датасету: (8068, 11)
Пропуски до обробки:
 ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64


In [32]:
if 'ID' in df.columns:
    df = df.drop(columns=['ID'])

X = df.drop(columns=['Segmentation'])
y = df['Segmentation']

le = LabelEncoder()
y_encoded = le.fit_transform(y)


target_names = le.classes_
print(f"Класи закодовано: {target_names} -> [0 1 2 3]")

Класи закодовано: ['A' 'B' 'C' 'D'] -> [0 1 2 3]


In [33]:
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


X_train_processed = preprocessor.fit_transform(X_train)

X_test_processed = preprocessor.transform(X_test)

print(f"Розмір X_train (оброблений): {X_train_processed.shape}")
print(f"Розмір X_test (оброблений): {X_test_processed.shape}")

Розмір X_train (оброблений): (6454, 28)
Розмір X_test (оброблений): (1614, 28)


In [34]:
feature_names = preprocessor.get_feature_names_out()

display(pd.DataFrame(X_train_processed, columns=feature_names))


,num__Age,num__Work_Experience,num__Family_Size,cat__Gender_Female,cat__Gender_Male,cat__Ever_Married_No,cat__Ever_Married_Yes,cat__Graduated_No,cat__Graduated_Yes,cat__Profession_Artist,...,cat__Spending_Score_Average,cat__Spending_Score_High,cat__Spending_Score_Low,cat__Var_1_Cat_1,cat__Var_1_Cat_2,cat__Var_1_Cat_3,cat__Var_1_Cat_4,cat__Var_1_Cat_5,cat__Var_1_Cat_6,cat__Var_1_Cat_7
0,-0.695320,1.970880,-1.231118,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.703982,-0.456381,-0.564314,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.635337,-0.456381,0.769294,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.264401,-0.759788,2.102901,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.935250,1.970880,-1.231118,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6449,-0.995233,1.667472,-1.231118,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6450,-0.395407,-0.759788,-0.564314,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6451,-0.995233,1.060657,2.102901,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6452,-1.055215,-0.759788,-0.564314,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [2]:
!pip install imbalanced-learn

In [35]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import OrdinalEncoder

In [36]:
numeric_transformer_smote = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer_smote = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor_smote = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_smote, numeric_cols),
        ('cat', categorical_transformer_smote, categorical_cols)
    ],
    verbose_feature_names_out=False
).set_output(transform="pandas")

X_train_enc = preprocessor_smote.fit_transform(X_train)

all_cols = X_train_enc.columns

cat_feature_indices = [i for i, col in enumerate(all_cols) if col in categorical_cols]

print(f"Індекси категоріальних ознак: {cat_feature_indices}")
print(f"Приклад ознак за цими індексами: {[all_cols[i] for i in cat_feature_indices]}")

smote_nc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)

X_train_smotenc, y_train_smotenc = smote_nc.fit_resample(X_train_enc, y_train)

print(f"Розмір Train до: {X_train_enc.shape}, після SMOTENC: {X_train_smotenc.shape}")


smote_tomek_nc = SMOTETomek(smote=smote_nc, random_state=42)

X_train_st_nc, y_train_st_nc = smote_tomek_nc.fit_resample(X_train_enc, y_train)

print(f"Розмір Train після SMOTE-Tomek: {X_train_st_nc.shape}")

Індекси категоріальних ознак: [3, 4, 5, 6, 7, 8]
Приклад ознак за цими індексами: ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
Розмір Train до: (6454, 9), після SMOTENC: (7256, 9)
Розмір Train після SMOTE-Tomek: (6100, 9)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_test_enc = preprocessor_smote.transform(X_test)

def train_eval(X_train_data, y_train_data, X_test_data, y_test_data, name):
    clf = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=3000, random_state=42)

    clf.fit(X_train_data, y_train_data)

    y_pred = clf.predict(X_test_data)

    print(f"\n{'='*10} {name} {'='*10}")
    print(classification_report(y_test_data, y_pred, target_names=le.classes_))

train_eval(X_train_enc, y_train, X_test_enc, y_test, "ORIGINAL (Ordinal)")

train_eval(X_train_smotenc, y_train_smotenc, X_test_enc, y_test, "SMOTE (Ordinal)")

train_eval(X_train_st_nc, y_train_st_nc, X_test_enc, y_test, "SMOTE-TOMEK (Ordinal)")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



========== ORIGINAL (Ordinal) ==========
              precision    recall  f1-score   support

           A       0.39      0.39      0.39       394
           B       0.41      0.08      0.14       372
           C       0.47      0.64      0.54       394
           D       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.44      1614
weighted avg       0.47      0.49      0.45      1614



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



========== SMOTE (Ordinal) ==========
              precision    recall  f1-score   support

           A       0.41      0.40      0.41       394
           B       0.37      0.14      0.21       372
           C       0.48      0.63      0.54       394
           D       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.49      0.46      1614
weighted avg       0.48      0.50      0.47      1614



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



========== SMOTE-TOMEK (Ordinal) ==========
              precision    recall  f1-score   support

           A       0.40      0.35      0.37       394
           B       0.36      0.17      0.23       372
           C       0.48      0.62      0.54       394
           D       0.61      0.78      0.68       454

    accuracy                           0.50      1614
   macro avg       0.46      0.48      0.46      1614
weighted avg       0.47      0.50      0.47      1614



1. Напишіть, яку метрику ви обрали для порівняння моделей.

Для порівняння моделей я обрала метрику Macro Avg F1-score.
Чому:

Наші класи не є ідеально збалансованими (клас D має 454 приклади, а клас B — лише 372). Використання звичайної Accuracy було б некоректним, оскільки модель могла б ігнорувати малі класи і все одно мати непогану точність.

Macro Avg F1 усереднює показники всіх класів, надаючи їм однакову вагу. Це дозволяє побачити, чи почала модель краще розпізнавати складні класи (наприклад, B), а не лише найпопулярніші.

2. Яка модель найкраща?

Найкращий результат показала модель з використанням SMOTE (або SMOTE-Tomek).
Хоча загальна точність (Accuracy) зросла несуттєво (з 0.49 до 0.50), метрики для "проблемного" класу B покращилися помітно:

Original: F1-score для класу B був 0.14.

SMOTE: F1-score для класу B виріс до 0.21.

SMOTE-Tomek: F1-score для класу B виріс до 0.23 (найкращий результат).

Отже, балансування допомогло моделі "побачити" рідкісний клас, хоча загальна якість моделі залишається невисокою.

3. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

Різниця між Original (Macro F1 0.44) і SMOTE (Macro F1 0.46) є, але вона не кардинальна. Моя гіпотеза, чому ми не отримали стрибка якості до 0.8-0.9:

Лінійність моделі: Ми використовуємо Логістичну Регресію, яка намагається розділити класи прямими лініями. Ймовірно, залежність між ознаками (Вік, Професія) та сегментом клієнта є нелінійною і складною. Просте додавання синтетичних даних (SMOTE) не допомагає провести пряму лінію там, де потрібна крива.

Перетин класів (Overlap): Судячи з низької точності навіть на тренувальних даних, класи сильно перемішані між собою. Клієнти з сегмента A і сегмента B можуть мати майже однаковий вік і професію. У такій ситуації SMOTE створює нові точки прямо "поверх" чужого класу, що додає шуму, а не допомагає розділенню.

Слабкі ознаки: Можливо, наявних ознак (стать, вік, стаж) просто недостатньо, щоб чітко розрізнити ці 4 маркетингові сегменти. Потрібні додаткові дані або більш потужна модель (наприклад, Random Forest).